<a href="https://colab.research.google.com/github/l2yao/translator/blob/main/Translate_a_Public_Domain_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Translate a public domain book

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Translate_a_Public_Domain_Book.ipynb"><img src = "../images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

In this notebook, you will explore Gemini model as a translation tool, demonstrating how to prepare data, create effective prompts, and save results into a `.txt` file.

This approach significantly improves knowledge accessibility. Various sources provide open-source books. In this notebook, you will use [Project Gutenberg](https://www.gutenberg.org/) as a resource.


This platform provides access to a wide range of books available for free download in PDF, eBook, TXT formats and more.

In [1]:
!pip install -U -q "google-generativeai>=0.7.2" tqdm

In [2]:
import google.generativeai as genai

from tqdm import tqdm
from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Data preparation

You will translate a well-known book written by Arthur Conan Doyle from Polish (trans. Eugenia Żmijewska) to English about the detective Sherlock Holmes. Here are the titles in Polish and English:



* Polish Title: *Tajemnica Baskerville'ów: dziwne przygody Sherlocka Holmes*

* English Title: *The Hound of the Baskervilles*

In [4]:
with open("book1.txt") as f:
      book = f.read()

Books contain all sorts of fictional or historical descriptions, some of them rather literal and might cause the model to stop from performing translation query. To prevent some of those exceptions users are able to change `safety_setting` from default to more open approach.

Go ahead and initialize the Gemini 1.5 Flash model for this task.

In [5]:
model = genai.GenerativeModel('gemini-2.0-flash')

def generate_output(prompt):
    response = model.generate_content(
        prompt,
        request_options = {
            "timeout": 1000
        }
    )

    try:
        return response.text
    except Exception as ex:
        raise ex

Begin by counting how many tokens are in the book to know if you need to proceed with the next step (split text into smaller chunks).

## Token Information

LLMs are constrained by token limits. In order to stay within the limit of Gemini 1.5 Flash, it's important to ensure the following:

* Input token limit should not exceed: 1,048,576 tokens
* Output token limit should not exceed: 8,192 tokens

Since input chunks of post-translation text will serve as outputs, it's advisable to split them into smaller segments.

As you will see in the example, there are over 1000 small chunks. Calling API for the token count of each one would take a long time. Because of this estimate 1 token is equal to 4 characters will be used.

To account for the possible miscalculation of tokens and output potentially being longer than input each chunk will use up to 4*5000 characters. Remember to adjust the maximum to your needs.

For more details on tokens in Gemini models, refer to the documentation [here](https://ai.google.dev/gemini-api/docs/models/gemini
).

### Split text into chunks
Adjust the number of whitespace characters used for splitting based on the book you are working with.

Then, ensure to consider token limitations to receive complete responses from the model.


In [6]:
chunks = book.split("\n\n")

num_chunks = len(chunks)
print(f"Number of chunks: {num_chunks}")

Number of chunks: 2


In [7]:
estimated_token_counts = []

for chunk in chunks:
    estimated_token_count = len(chunk)
    estimated_token_counts.append(estimated_token_count)

# You can print number of estimated tokens in each non empty chunk to see what are you working with
print(estimated_token_counts)

[18, 31700]


# Identify the largest number of tokens in a chunk to check if any exceed your token limitations. In this case, the chunks are very small, so this should not be an issue.

If you encounter this problem, consider splitting the text into even smaller chunks.

In [8]:
max(estimated_token_counts)

31700

### Group smaller chunks together
As observed above, certain chunks contain very few tokens. While the LLM can translate these sentences accurately, providing larger context can enhance its understanding of the content.

Let's combine smaller chunks into larger ones for improved results.

In [9]:
def split_chunk(chunk, max_len=6000):
    result = []
    index = 0
    while index < len(chunk):
      start = index
      if index + max_len < len(chunk):
        end = chunk.rfind("。", index, index + max_len) + 1
      else:
        end = len(chunk)
      result.append(chunk[start:end])
      index = end

    return result

def chunk_splitting(chunks, max_len=6000):
    result = []
    for chunk in chunks:
      result.extend(split_chunk(chunk, max_len))
    return result

In [10]:
# Combine short chunks into larger groups based on token counts.
def chunk_grouping(chunks, token_counts, max_len=6000):
    grouped_chunks = []
    current_group = ""
    current_token_sum = 0

    # Process each chunk and group them based on token limits
    for index in range(len(chunks)):
        chunk = chunks[index]
        count = token_counts[index]
        if token_counts[index] == 0:
            continue

        current_group += "\n\n" + chunk
        current_token_sum += count  # Count in 1 token for newlines
        # Add a new chunk if there is space available in the current group.
        if current_token_sum + 1 + count > max_len:
            grouped_chunks.append(current_group)
            current_group = ""
            current_token_sum = 0


    return grouped_chunks


grouped_chunks = chunk_grouping(chunks, estimated_token_counts)

final_chunks = chunk_splitting(grouped_chunks)

The last step of splitting is checking if the non-estimated token counts are acceptable.

In [11]:
token_counts = []
for chunk in final_chunks:
  response = model.count_tokens(chunk,
                                request_options = {'timeout': 600})
  token_count = response.total_tokens
  token_counts.append(token_count)
token_counts

[5088, 5576, 5579, 5590, 5335, 1486]

Despite using estimates, this split minimizes the calls necessary to translate the book well, and most chunks are of similar token counts to each other.

## Translation

Your book is ready for translations! The selected book has been divided into 13 larger chunks.

### Prompt

Specify the original language of your book and the language you would like it to be translated into.


In [12]:
def translations(text,
                 inputLanguage='Chinese',
                 targetLanguage='English'):

    translation_prompt = f"""
    As a professional book translator,
    translate the following book from {inputLanguage} into {targetLanguage}.

    Only return the translation please.
    
    Book to Translate:
    {text}

    """

    return translation_prompt

### Translate chunks

The translation time depends on the amount of text being processed. Our selected book was 318 KB and the entire translation took 20 minutes.

Please note that larger texts may take several hours to complete.

In [14]:
results = []
index = 0
translation = None

for chunk in tqdm(final_chunks):
    try:
        translation = generate_output(translations(chunk))
        results.append(translation)
    except Exception as ex:
        print(f"\nChunk was not included in translation due to possible sensitive content. Text: {chunk}")

    index += 1

100%|██████████| 6/6 [04:30<00:00, 45.11s/it]


All done, now you can print the chunk before and after translation to see the results.

You can see, that translation effectively maintains the narrative's novel quality, proper grammar and sentence structure.

Next step will be joining chunks together and saving the translated book.

In [13]:
Markdown(final_chunks[0])



﻿修行语录
 
高雄净宗学会辑录
 


 
缘起
 
佛法是人世间最伟大的宝藏，佛陀将宇宙人生的真相，因果报应的原理与追求健康幸福美满的人生，解脱六道轮回，了生脱死的方法，以究竟圆满，无上正等正觉的觉悟者的实证体认，全盘托出，教育一切后学，在现世得到安乐，临终得以解脱。
 
然而，佛法从二、三年千年前在印度的正法时代，到东汉传入中国以后，进入了像法时代，到明末清初，进入了末法时代，佛陀早已预见了佛教的法运将会日趋衰败，因为末法时代人心更败坏，伦理更浇薄，天地世间的乱象将愈演愈烈，佛门的弊病更将每况愈下。然而，佛陀的教导并没有问题，佛法的精神与本质一点也没有改变，只是末法时代的众生业深障重，福薄慧浅，根机陋劣，弘法的人不识根机，教学有问题，学佛的人盲修瞎练，学习有问题，甚至邪魔外道混迹佛门，以似是而非的佛法，行沽名钓誉，搜括敛财之实，末法时代，邪师说法，如恒河沙，学佛人稍不小心，就容易悖离了佛教的精神与本质，掉入了邪恶的陷阱，沦为魔子魔孙而不自知。
 
末法时代善知识难逢，明师难遇，到处是外表光鲜，言行不一的邪师，到处是以盲引盲，招徒蓄众的邪魔，学佛人如果不能明辨正邪，有所定见，就容易人云亦云，沾染魔见，走火入魔，陷自己于万劫不复的地步，这一点大家千万不可轻忽。
 
其实，末法时代学佛最安全最保险的方法就是—‘以古为师’，以志节高超，名耀千古的历代祖师大德为师，以祖师大德的珍贵开示作为我们修行的座右铭。而距离我们这个时代最近，又最受大众景仰的首推—印光大师，因此，‘印光大师全集’、‘印光大师文钞菁华录’、‘印光大师全集问答撷录’等书，为有志学佛者当奉为圭臬，日日温习的宝典。常读印光大师文钞并依之努力实践，即是常亲近印光大师，即是印光大师的入门弟子，如此学佛，必能日起有功，稳操胜算！
 
这本‘修行语录’是希望透过撷录佛陀在经典中开示的最要精华，配合历代祖师大德教诲的最要叮咛，让末法时代的吾辈陋劣凡夫，能在最短的时间内掌握住佛法的修行要点，依之实践，即可跟随佛陀与历代祖师的步伐，成功地走向安乐解脱的康庄大道！
 
修行十大要点
 
一、孝养父母
 
佛教以孝为本，孝养父母是佛门的最大根本，也是做人最基本的要件，试想连乌鸦尚知反哺报恩，如果我们不能孝养父母，岂不是连禽兽也不如吗？
 
父母是我们一生中最大的福田，孝养父母与孝养佛的功德无二无别，不能孝养父母是人生最大的过错，不孝之人是一点也谈不上学佛的资格。
 
此外，劝父母信愿念佛，求生西方，永脱轮回生死之苦才是最究竟最圆满的孝道。
 
二、恪尽本分
 
人生世间，每个人都有自己应尽的本分与职责，学佛人应当在自己的工作岗位上精进努力，做家庭、社会、国家的模范榜样，能孝顺父母，教育子女，爱护家庭，利益社会，报效国家，方是真学佛人，逃避了自己应尽的职责，不能恪尽本分，侈言学佛，都是自欺欺人，学佛不可能有所成就。
 
三、深信因果
 
整个佛法的核心就是‘因果’二字，种了善因必得善果，种了恶因必得恶果，因果报应，丝毫不爽，不是不报，时候未到。学佛人当深信因果，以戒为师，日日反省，随时改过。
 
此外，念佛人深信种了念佛的善因，必得成佛的善果，这是最最深奥的因果。
 
四、戒杀吃素
 
学佛人当诸恶莫作，将一切的罪恶行为努力断除。而所有的罪恶中，以杀生吃肉的罪恶最重，因为众生最宝贵的，无非自己的生命，杀它吃它，它最恼恨，怨仇结得最深，将来杀债杀报，血债血还，恶果最为惨烈，因此，诸恶莫作中以戒杀吃素为急！
 
五、放生救命
 
学佛人当众善奉行，任何的善事，只要有机会当努力实践。而所有的善行中，放生第一，因为放生是救急救命的行为，功德最大，非其他小善所能比拟。而众生最宝贵的，无非自己的生命，放它救它，它最感激，善缘结得最深，未来福德善报最不可思议，所以，众善奉行中以放生救命为先！
 
六、竭诚尽敬
 
诚心敬意是成就天下任何事业的基础，印光大师更教导我们要竭心尽力地把诚敬二字做到了极点，有一分诚敬，就有一分功德，有十分诚敬，就有十分功德，这是学佛成功与否的绝妙秘诀，大家绝对要谨记在心！
 
七、发菩提心
 
学佛的功德大小与自己的心量成正比，心量有多大，功德就有多大。因此学佛人的心量一定要大！做任何事绝不是为了自私自利！一定要从内心发真诚心，真实为利益一切众生。我们上求佛道，发心成佛后有能力要普渡众生；我们下化众生，发心有机缘一定要把佛法最精要的重点传播大众。
 
此外，真正愿意念佛求生西方极乐世界的心，就是真正的发菩提心。
 
八、拜佛忏悔
 
我们累世以来所造作的罪业，若是有体有相，是尽虚空也装不下，我们其实只不过是个业障深重的浊恶凡夫罢了。因此，学佛人当发惭愧心及忏悔心，勤行礼佛，因为礼佛一拜，罪灭河沙，拜佛忏悔是表达我们诚心敬意，惭愧自省最好的方法。
 
九、信愿念佛
 
净土法门是依靠佛力救拔，信则信西方极乐世界有一尊阿弥陀佛，愿则愿意速生极乐世界，厌离此苦世界，行则老老实实执持一句‘南无阿弥陀佛’圣号，只要‘信愿行’三资粮具足，就可当生倚仗阿弥陀佛的大誓愿力救拔，超生脱死，永断轮回，是佛教一切法门中最深奥最奇妙最特别最方便的法门！
 
照理说，一句‘阿弥陀佛’是靠佛力救拔，是绝对绝对保证往生西方的，但为什么当令念佛的人多，而真正往生西方的人却很少呢？
 
这是一个非常严肃关键的问题，当今念佛人无法往生的关键在于—‘缺乏信愿，怕死，根本不愿往生。’当令念佛人念佛只是求长寿求健康求平安求富贵求顺遂，求世间一切利益，但绝不求往生西方！
 
‘死’之一字即可勘验念佛人是否具足信愿。扪心自问，你怕不怕死？如果现在马上得死，马上阿弥陀佛就接你往生西方，你愿不愿意？一个真正厌离娑婆，欣求极乐的真念佛人，一定是视死如归，随时愿意早点跟随阿弥陀佛到西方极乐世界去的，是绝对不怕死，马上愿意死，马上愿意随时往生的。至于表里不一，假信虚愿的假念佛人，则是贪生怕死，不愿意死，不愿往生，求多活几年，有太多太多的理由牵绊敷衍。要知道，只此一怕死，不愿死，不愿求往生之心，即与阿弥陀佛的心相违逆，信愿行三资粮不具足，如何成功地往生西方呢？
 
当今念佛人千有九百九十九多为假念佛人，‘死’之一字，当时时刻在心头，时时检验自己，只要念佛人不怕死，喜欢死，随时愿意死，随时愿意早一点往生极乐，就是真正的‘厌离娑婆，欣求极乐’的真念佛人，这种人千念千生，万念万生，绝无不生西方之人！
 
十、实践力行
 
佛法最重要的是实践，一个一字不识，完全不懂经教，只会吃素拜佛，老老实实执持一句‘阿弥陀佛’的老太婆，学佛的成就往往远胜于辩才无碍，精通三藏的大通家，关键即在于是否能够实践力行，能知不能行，犹如纸上谈兵，说食数宝，只是惘然！
 
总而言之，学佛但求专精，最忌夹杂，只要掌握住要点努力实践，就能真正得到佛法的实益。
 
一、净土五经菁华
 
佛说阿弥陀经
 
※从是西方过十万亿佛土。有世界名曰极乐。其土有佛。号阿弥陀。令现在说法。
 
※其国众生。无有众苦。但受诸乐。故名极乐。
 
※彼佛国土。无三恶道。舍利弗。其佛国土。尚无恶道之名。何况有实。
 
※彼佛光明无量。照十方国。无所障碍。是故号为阿弥陀。
 
※彼佛寿命。及其人民。无量无边阿僧祇劫。故名阿弥陀。
 
※阿弥陀佛成佛以来。于今十劫。
 
※极乐国土。众生生者。皆是阿鞞跋致。
 
※众生闻者。应当发愿。愿生彼国。所以者何。得与如是诸上善人。俱会一处。
 
※不可以少善根福德因缘。得生彼国。
 
※若有善男子。善女人。闻说阿弥陀佛。执持名号。若一日。若二日。若三日。若四日。若五日。若六日。若七日。一心不乱。其人临命终时。阿弥陀佛。与诸圣众。现在其前。是人终时。心不颠倒。即得往生阿弥陀佛极乐国土。
 
※若有众生。闻是说者。应当发愿。生彼国土。
 
※汝等众生。当信是称赞不可思议功德。一切诸佛所护念经。
 
※若有善男子。善女人。闻是经受持者。及闻诸佛名者。是诸善男子。善女人。皆为一切诸佛之所护念。皆得不退转于阿耨多罗三藐三菩提。
 
※汝等皆当信受我语。及诸佛所说。
 
※若有人。已发愿。今发愿。当发愿。欲生阿弥陀佛国者。是诸人等。皆得不退转于阿耨多罗三藐三菩提。于彼国土。若已生。若今生。若当生。
 
※诸善男子。善女人。若有信者。应当发愿。生彼国土。
 
※当知我于五浊恶世。行此难事。得阿耨多罗三藐三菩提。为一切世间说此难信之法。是为甚难。
 
佛说无量寿经
 
※阿弥陀佛四十八大愿
 
一、国无恶道愿。        二、不堕恶趣愿。
三、身悉金色愿。        四、三十二相愿。
五、身无差别愿。        六、宿命通愿。
七、天眼通愿。          八、天耳通愿。
九、他心通愿。          十、神足通愿。
十一、遍供诸佛愿。      十二、定成正觉愿。
十三、光明无量愿。      十四、触光安乐愿。
十五、寿命无量愿。      十六、声闻无数愿。
十七、诸佛称叹愿。      十八、十念必生愿。
十九、闻名发心愿。      二十、临终接引愿。
二十一、悔过得生愿。    二十二、国无女人愿。
二十三、厌女转男愿。    二十四、莲华化生愿。
二十五、天人礼敬愿。    二十六、闻名得福愿。
二十七、修殊胜行愿。    二十八、国无不善愿。
二十九、住正定聚愿。    三十、乐如漏尽愿。
三十一、不贪计身愿。    三十二、那罗延身愿。
三十三、光明慧辩愿。    三十四、善谈法要愿。
三十五、一生补处愿。    三十六、教化随意愿。
三十七、衣食自至愿。    三十八、应念受供愿。
三十九、庄严无尽愿。    四十、无量色树愿。
四十一、树现佛刹愿。    四十二、彻照十方愿。
四十三、宝香普熏愿。    四十四、普等三昧愿。
四十五、定中供佛愿。    四十六、获陀罗尼愿。
四十七、闻名得忍愿。    四十八、现证不退愿。
 
※我作佛时。十方众生。闻我名号。至心信乐。所有善根。心心回向。愿生我国。乃至十念。若不生者。不取正觉。唯除五逆。诽谤正法。
 
※常运慈心拔有情。度尽无边苦众生。
 
※善护口业。不讥他过。
善护身业。不失律仪。
善护意业。清净无染。
 
※现在西方。去阎浮提百十俱胝那由他佛刹。有世界名曰极乐。法藏成佛。号阿弥陀。成佛以来、于今十劫。今现在说法。
 
※阿弥陀佛威神光明。最尊第一。十方诸佛。所不能及。
 
※惟阿弥陀佛。光明普照无量无边无数佛刹。
 
※阿弥陀佛。光明善好。胜于日月之明。十亿万倍。光中极尊。佛中之王。
 
※十方世界诸往生者。皆于七宝池莲华中。自然化生。悉受清虚之身。无极之体。不闻三涂恶脑苦难之名。尚无假设。何况实苦。但有自然快乐之音。是故彼国名为极乐。
 
※发菩提心。一向专念。阿弥陀佛。修诸功德。愿生彼国。此等众生。临寿终时。阿弥陀佛。与诸圣众。现在其前。经须臾间。即随彼佛往生其国。
 
※其有众生欲于今世见阿弥陀佛者。应发无上菩提之心。复当专念极乐国土。积集善根。应持回向。由此见佛。生彼国中。得不退转。乃至无上菩提。
 
※当发无上菩提之心。一向专念。阿弥陀佛。
 
※要当孝顺。至诚忠信。当信佛经语深。当信作善得福。
 
※诸往生者。皆得阿惟越致。皆具金色三十二相。皆当作佛。
 
※无量无数不可思议无有等等无边世界。诸佛如来。皆共称赞无量寿佛所有功德。
 
※其佛本愿力。闻名欲往生。自致不退转。
 
※世间善男子。善女人。若有急难恐怖。但自归命观世音菩萨。无不得解脱者。
 
※八方上下。佛国无数。阿弥陀国。长久广大。明好快乐。最为独胜。
 
※诸佛密藏。究竟明了。
 
※世人共争不急之务。于此剧恶极苦之中。勤身营务。以自给济。尊卑。贫富。少长。男女。累念积虑。为心走使。无田忧田。无宅忧宅。眷属财物。有无同忧。
 
※人在爱欲之中。独生独死。独去独来。苦乐自当。无有代者。
 
※何不于强健时。努力修善。欲何待乎。
 
※惑道者众。悟道者少。各怀杀毒。恶气冥冥。为妄兴事。违逆天地。
 
※爱欲荣华。不可常保。皆当别离。无可乐者。当勤精进。生安乐国。
 
※天地之间。自然有是。虽不即时报应。善恶会当归之。
 
※天地之间。五道分明。善恶报应。祸福相承。身自当之。无谁代者。
 
※云何第一。当自端心。当自端身。耳目口鼻。皆当自端。身心净洁。与善相应。
 
※当知疑惑。于诸菩萨为大损害。为失大利。是故应当明信诸佛无上智慧。
 
※若以无相智慧。植众德本。身心清净。远离分别。求生净刹。趣佛菩提。当生佛刹。永得解脱。
 
※若有善男子。善女人。得闻阿弥陀佛名号。能生一念喜爱之心。归依瞻礼。如说修行。当知此人为得大利。当获如上所说功德。心无下劣。亦不贡高。成就善根。悉皆增上。当知此人非是小乘。于我法中。得名第一弟子。
 
佛说观无量寿佛经
 
※欲生彼国者。当修三福。
 
一者。孝养父母。奉事师长。慈心不杀。修十善业。
二者。受持三归。具足众戒。不犯威仪。
三者。发菩提心。深信因果。读诵大乘。劝进行者。
 
如此三事。名为净业。
 
此三种业。乃是过去未来现在。三世诸佛。净业正因。
 
※诸佛如来。是法界身。遍入一切众生心想中。是故汝等心想佛时。是心即是三十二相。八十随形好。是心作佛。是心是佛。
 
※佛心者。大慈悲是。以无缘慈。摄诸众生。
 
※见无量寿佛者。即见十方无量诸佛。
 
※称南无阿弥陀佛。称佛名故。于念念中除八十亿劫生死之罪。
 
※若善男子及善女人。但闻佛名。二菩萨名。除无量劫生死之罪。何况忆念。若念佛者。当知此人。则是人中分陀利华。
 
※汝好持是语。持是语者。即是持无量寿佛名。
 
楞严经大势至菩萨念佛圆通章
 
※若众生心。忆佛念佛。现前当来。必定见佛。去佛不远。不假方便。自得心开。
 
※佛问圆通。我无选择。都摄六根。净念相继。得三摩地。斯为第一。
 
华严经普贤菩萨行愿品
 
※普贤菩萨十大愿王
 
一者。礼敬诸佛。        二者。称赞如来。
三者。广修供养。        四者。忏悔业障。
五者。随喜功德。        六者。请转法轮。
七者。请佛住世。        八者。常随佛学。
九者。恒顺众生。        十者。普皆回向。
 
※忏悔业障者。菩萨自念。我于过去无始劫中。由贪嗔痴。发身口意。作诸恶业。无量无边。若此恶业有体相者。尽虚空界。不能容受。我今悉以清净三业。遍于法界极微尘刹。一切诸佛菩萨众前。诚心忏悔。后不复造。恒住净戒。
 
※菩萨若能随顺众生。则为随顺供养诸佛。若于众生尊重承事。

In [15]:
Markdown(results[0])

Okay, here is a translation of the provided text, aiming for accuracy and readability, while maintaining a tone appropriate for a Buddhist text:

**A Compilation of Exhortations on Practice**

Collected by the Kaohsiung Pure Land Society

**Foreword**

The Buddha Dharma is the most magnificent treasure in the human world. The Buddha, through the ultimate, perfect, unsurpassed, and equitably enlightened realization of an awakened being, revealed the truths of the universe and human life, the principles of karmic cause and effect, and the methods for pursuing a healthy, happy, and fulfilling life. These methods lead to liberation from the six realms of transmigration and the cycle of birth and death. He fully laid out this teaching for all future generations to find peace in the present and liberation at the end of life.

However, from the Dharma's golden age in India two or three thousand years ago to its introduction to China during the Eastern Han Dynasty, which marked the beginning of the Semblance Dharma Age, and then to the late Ming and early Qing Dynasties, when it entered the Degenerate Dharma Age, the Buddha had long foreseen that the Dharma's fortune would gradually decline. This was because, in the Degenerate Dharma Age, people’s hearts would become increasingly corrupt, ethics more shallow, the world's chaos would intensify, and the maladies within Buddhist communities would continually worsen. Nonetheless, the Buddha's teachings are flawless. The spirit and essence of the Dharma have not changed. It is simply that beings in the Degenerate Dharma Age have deep karmic hindrances, meager blessings, shallow wisdom, and inferior faculties. Those who propagate the Dharma fail to understand the needs of beings, teachings are flawed, practitioners blindly practice, and learning is deficient. Even demonic and heretical paths infiltrate Buddhist communities, using distorted Dharma to seek fame, gain, and amass wealth. In the Degenerate Dharma Age, heretical teachers appear like the sands of the Ganges. If practitioners are not careful, they can easily stray from the spirit and essence of Buddhism, fall into evil traps, and become unwitting servants of demons.

In the Degenerate Dharma Age, it is difficult to encounter good and wise teachers and even harder to meet enlightened masters. Everywhere there are outwardly radiant but inconsistent heretical teachers, and everywhere there are the blind leading the blind, gathering disciples like demons. If practitioners cannot distinguish right from wrong and lack steadfast views, they easily follow the crowd, absorb demonic views, go astray, and plunge themselves into eternal suffering. This is a matter that should never be taken lightly.

In fact, the safest and most reliable method for practicing Buddhism in the Degenerate Dharma Age is to ‘take the ancients as our teachers.’ We should take the great masters of past generations, with their lofty aspirations and renowned achievements, as our guides and their precious teachings as the mottos for our own practice. Among those closest to our time and most revered by the masses, Master Yin Guang stands out. Therefore, the ‘Complete Works of Master Yin Guang,’ the ‘Selected Excerpts from Master Yin Guang's Essays,’ and the ‘Selected Q&A from the Complete Works of Master Yin Guang’ are the canon that those aspiring to learn Buddhism should respect and review daily. By constantly reading Master Yin Guang's writings and striving to put them into practice, we are constantly drawing near to him and becoming his direct disciples. Through this kind of practice, we can be certain to achieve daily progress and success!

This book, “Exhortations on Practice,” aims to extract the most essential insights from the Buddha’s sutras, combined with the most important instructions from the teachings of great masters, so that we ordinary beings of the Degenerate Dharma Age can quickly grasp the key points of Buddhist practice. By putting them into practice, we can successfully follow in the footsteps of the Buddha and past masters and embark on the broad path to peace and liberation!

**Ten Essential Points for Practice**

**1. Filial Piety Towards Parents**

Buddhism regards filial piety as the foundation. Filial piety toward parents is the greatest root of the Buddhist path and the most basic element of being human. Even crows know how to repay their parents’ kindness. If we cannot be filial to our parents, are we not worse than beasts?

Our parents are the greatest fields of merit in our lives. The merit of being filial to parents is no different from that of being filial to the Buddha. The inability to be filial to parents is the greatest mistake in life. Those who are unfilial do not even have the basic qualifications to practice Buddhism.

Furthermore, persuading our parents to have faith, vow to be reborn in the West, and be forever liberated from the suffering of transmigration is the most ultimate and complete form of filial piety.

**2. Fulfilling One's Duties**

In this world, each person has their own duties and responsibilities. Practitioners of Buddhism should be diligent in their work, setting an example for their families, society, and nation. Being filial to parents, educating children, caring for the family, benefiting society, and serving the country – these are the marks of a true practitioner. To avoid one's responsibilities and fail to fulfill one's duties while claiming to be a practitioner is self-deception. One will not achieve anything in this way.

**3. Deep Faith in Karma**

The heart of all Buddhist teachings is the principle of cause and effect. Planting good causes will surely lead to good results, and planting evil causes will surely lead to evil results. The law of karma is infallible. If retribution is not seen, it's merely because the time is not yet ripe. Practitioners of Buddhism should deeply believe in karma, take the precepts as their teacher, reflect daily, and constantly correct their faults.

Furthermore, those who practice mindfulness of the Buddha deeply believe that planting the good seed of Buddha recitation will surely lead to the good result of Buddhahood. This is the most profound of karmic principles.

**4. Abstaining from Killing and Eating Meat**

Practitioners of Buddhism should avoid all evil deeds, making every effort to cut off all sinful behavior. Among all sins, the sin of killing and eating meat is the gravest. This is because what beings value most is their own lives. To kill them and eat them is the source of their deepest resentment, creating the deepest karmic debts. In the future, these debts will surely be repaid with blood and suffering. Therefore, among all evils, abstaining from killing and eating meat is the most urgent!

**5. Releasing Lives and Saving Beings**

Practitioners of Buddhism should practice all good deeds, making every effort to act whenever opportunities arise. Among all good acts, releasing lives is paramount. This is because it is an act of emergency rescue, with the greatest merit, far surpassing other minor acts of good. What beings value most is their own lives. To release and save them elicits their deepest gratitude, establishing the deepest good karmic connections. Future blessings will be incredible. Therefore, among all good deeds, releasing lives and saving beings should be prioritized!

**6. Utmost Sincerity and Reverence**

Sincerity and respect are the foundations for accomplishing any endeavor in the world. Master Yin Guang further teaches that we should exert ourselves to the utmost, putting sincerity and respect into practice in the most extreme way. With a bit of sincerity and respect, we gain a bit of merit. With full sincerity and respect, we gain full merit. This is the miraculous secret to success in Buddhist practice that everyone must remember!

**7. Awakening the Bodhi Mind**

The merit of Buddhist practice is directly proportional to the breadth of one's heart. The greater the heart, the greater the merit. Therefore, practitioners of Buddhism must have a vast heart. They should never do anything for selfish gain! They must truly awaken within their hearts a sincere desire to benefit all sentient beings. We aspire to achieve Buddhahood and, upon doing so, will have the ability to liberate all beings. We guide beings, aspiring to share the essence of the Dharma with the masses whenever the opportunity arises.

Furthermore, the sincere desire to be reborn in the Western Pure Land is the true awakening of the Bodhi Mind.

**8. Bowing to the Buddha and Repenting**

The sins we have committed throughout countless lifetimes, if they had a physical form, would be more vast than empty space itself. We are, in reality, but ordinary beings burdened with deep karmic hindrances. Therefore, practitioners of Buddhism must cultivate a sense of shame and repentance and diligently perform prostrations. One prostration can eliminate countless sins. Prostrating and repenting is the best way to express our sincerity, respect, and self-reflection.

**9. Faith, Vow, and Mindfulness of the Buddha**

The Pure Land path relies on the Buddha’s power for salvation. Faith means believing that there is an Amitabha Buddha in the Western Pure Land. Vow means sincerely wanting to be reborn in that land and to leave this suffering world. Practice means to wholeheartedly recite the holy name “Namo Amitabha Buddha.” As long as one possesses the three provisions of faith, vow, and practice, one can, in this very lifetime, rely on Amitabha Buddha's great vows for salvation, escape the cycle of birth and death, and end transmigration forever. This is the deepest, most wondrous, most special, and most convenient of all Buddhist paths!

Logically speaking, a single recitation of ‘Amitabha Buddha’ relies on the Buddha's power for salvation and is a completely, absolutely guaranteed way to be reborn in the Western Pure Land. However, why are there so many people reciting the Buddha’s name now, yet so few actually reborn in the Pure Land?

This is a very serious and critical issue. The reason why people reciting the Buddha's name today fail to be reborn is that they ‘lack faith and vow, fear death, and do not genuinely desire rebirth.’ Most people recite the Buddha’s name simply to seek longevity, health, peace, wealth, and worldly success. They absolutely do not seek rebirth in the Western Pure Land!

The single word ‘death’ can be used to test whether a practitioner of mindfulness of the Buddha possesses genuine faith and vow. Ask yourself, are you afraid of death? If you had to die now, and Amitabha Buddha was ready to receive you into the Western Pure Land, would you be willing? A true practitioner who detests this Saha world and rejoices in the Pure Land would surely view death as going home and be willing to go with Amitabha Buddha to the Western Pure Land as soon as possible. They would absolutely not be afraid of death. They would welcome death and be ready to be reborn at any time. However, the hypocritical and falsely devoted practitioners are attached to life and afraid of death. They do not want to die or be reborn. They make excuses and cling to life. They want to live a few more years. Know this: this single fear of death, this unwillingness to die, this unwillingness to be reborn is contrary to Amitabha Buddha's heart. If one lacks the three provisions of faith, vow, and practice, how can one successfully be reborn in the Western Pure Land?

Most people reciting the Buddha's name today are, in fact, false practitioners. The word 'death' must be constantly kept in mind so we can test ourselves. As long as a practitioner is not afraid of death, welcomes death, and is ready to be reborn in the Pure Land as soon as possible, they are truly a person who "detests the Saha world and rejoices in the Pure Land." Such a person, whether they recite the Buddha's name one thousand times or ten thousand times, will surely be reborn in the Pure Land!

**10. Practice and Implementation**

The most important thing in Buddhism is practice. An old woman who is illiterate, who doesn't understand the scriptures, who only knows to eat vegetarian food, prostrate before the Buddha, and sincerely recite 'Amitabha Buddha' often achieves more than a renowned scholar who is eloquent and has mastered the three canons. The key is whether one is able to practice and implement. Knowledge without action is like fighting battles on paper or counting treasures in a dream. It is ultimately futile!

In conclusion, in the study of Buddhism, one should strive for focus and avoid distractions. As long as one grasps the key points and strives to implement them, one will truly realize the practical benefits of the Dharma.

**II. Essence of the Five Pure Land Sutras**

**The Amitabha Sutra (Buddha Speaks of Amitabha)**

*   “From here, passing over hundreds of thousands of millions of Buddha lands to the West, there is a world called ‘Ultimate Bliss’. In that land, there is a Buddha named Amitabha, now teaching the Dharma.”
*   “The sentient beings of that country experience no suffering, but only various joys. Hence, it is called ‘Ultimate Bliss’.”
*   “In that Buddha land, there are no three evil paths. Shariputra, in that Buddha land, even the names of evil paths do not exist, let alone the reality.”
*   “The Buddha's light is immeasurable, illuminating all the lands of the ten directions without obstruction. Therefore, He is called Amitabha.”
*   “The Buddha's lifespan and that of His people is immeasurable and without limit of Asankheya Kalpas. Therefore, He is called Amitabha.”
*   “Since Amitabha Buddha attained Buddhahood, ten kalpas have passed until now.”
*   “In the land of Ultimate Bliss, all those born are non-retrogression (Avivartika).”
*   “Sentient beings, upon hearing this, should make a vow to be born in that land. Why is that? Because they will be able to meet and be together with all the supreme sages.”
*   “One cannot be born in that land with a small amount of roots of goodness, merits, and causes.”
*   “If a good man or good woman, hearing of Amitabha Buddha, hold onto His name, if for one day, two days, three days, four days, five days, six days, or seven days, with focused concentration, at the time of their death, Amitabha Buddha and His assembly of sages will appear before them. At the time of death, their minds will not be confused, and they will be born in the land of Ultimate Bliss of Amitabha Buddha.”
*   “If there are sentient beings who hear these words, they should make the vow to be born in that land.”
*   “You sentient beings, should believe in the praise of the inconceivable merit of this sutra that is protected by all Buddhas.”
*   “If a good man or good woman, upon hearing this sutra, accepts it, upholds it, and hears the names of the Buddhas, all these good men and women will be protected by all the Buddhas, and they shall all attain non-retrogression from Anuttara-Samyak-Sambodhi.”
*   “You all should believe and accept my words and the words of all the Buddhas.”
*   “If there is anyone who has already made a vow, is making a vow, or will make a vow to be born in Amitabha Buddha’s land, all of these people will attain non-retrogression from Anuttara-Samyak-Sambodhi. In that land, whether they have already been born, are being born now, or will be born, all will attain enlightenment.”
*   “All good men and good women, if they have faith, should make the vow to be born in that land.”
*   “Know that I, in this world of five turbidities, have accomplished this difficult thing and attained Anuttara-Samyak-Sambodhi and have taught this difficult-to-believe Dharma to all sentient beings. This is truly difficult.”

**The Infinite Life Sutra (Buddha Speaks of Immeasurable Life)**

*   **The Forty-Eight Great Vows of Amitabha Buddha**
    1.  Vow of No Evil Paths in the Land
    2.  Vow of Not Falling into Evil Realms
    3.  Vow of Golden Bodies
    4.  Vow of Thirty-Two Marks
    5.  Vow of Undifferentiated Bodies
    6.  Vow of Knowing Past Lives
    7.  Vow of Heavenly Vision
    8.  Vow of Heavenly Hearing
    9.  Vow of Knowing Others’ Minds
    10. Vow of Divine Powers
    11. Vow of Universal Offerings
    12. Vow of Achieving Buddhahood
    13. Vow of Immeasurable Light
    14. Vow of Blissful Touch of Light
    15. Vow of Immeasurable Lifespan
    16. Vow of Countless Hearers
    17. Vow of Praise by All Buddhas
    18. Vow of Sure Rebirth with Ten Recitations
    19. Vow of Faith and Vow upon Hearing the Name
    20. Vow of Receiving at Death
    21. Vow of Rebirth after Repentance
    22. Vow of No Women in the Land
    23. Vow of Transforming from Women to Men
    24. Vow of Being Born from Lotuses
    25. Vow of Respect from Gods and Humans
    26. Vow of Receiving Blessings upon Hearing the Name
    27. Vow of Cultivating Superior Practices
    28. Vow of No Evil in the Land
    29. Vow of Dwelling in the Rightly Established State
    30. Vow of Joy Like the Extinction of Leakages
    31. Vow of Not Clinging to the Body
    32. Vow of Narayana Body
    33. Vow of Light and Wisdom
    34. Vow of Skillful Dharma Teaching
    35. Vow of One Lifetime to Buddhahood
    36. Vow of Teaching at Will
    37. Vow of Self-Provided Clothing and Food
    38. Vow of Receiving Offerings on Demand
    39. Vow of Inexhaustible Adornment
    40. Vow of Infinite Number of Colorful Trees
    41. Vow of Trees Reflecting Buddha Lands
    42. Vow of Illumination of the Ten Directions
    43. Vow of Incense Permeating the Universe
    44. Vow of Universal Samadhi
    45. Vow of Offering to Buddhas in Samadhi
    46. Vow of Acquiring Dharani
    47. Vow of Obtaining Tolerance Upon Hearing the Name
    48. Vow of Immediate Non-Retrogression

*   “When I achieve Buddhahood, if sentient beings of the ten directions, upon hearing my name, wholeheartedly believe and rejoice, and dedicate all their roots of goodness, aspiring to be born in my land, even if with only ten recitations, if they are not reborn, then I will not achieve perfect enlightenment. The exception is for those who commit the five heinous crimes or slander the true Dharma.”
*   “Constantly use compassion to liberate sentient beings, completely liberating all suffering beings.”
*   “Carefully guard your speech, not criticizing others’ faults.
Carefully guard your body, not straying from the precepts.
Carefully guard your mind, with purity and no defilements.”
*   “To the West, past billions of Buddha lands, there is a world called Ultimate Bliss. The Dharma Treasury has become a Buddha named Amitabha. Since achieving Buddhahood, ten kalpas have passed until now. He is now teaching the Dharma.”
*   “The majesty and light of Amitabha Buddha is the most honored and supreme, beyond the reach of all Buddhas in the ten directions.”
*   “Only Amitabha Buddha's light illuminates limitless, boundless, and countless Buddha lands.”
*  “The light of Amitabha Buddha is good and magnificent, surpassing the light of the sun and moon by ten billion times. His light is the most revered, and He is the King among Buddhas.”
*   “All those who are reborn in the ten directions are born naturally in the seven-jeweled lotus ponds. They receive bodies of pure emptiness and infinite forms. They do not hear the names of the three evil paths. Even the concept of suffering is non-existent, let alone the actual suffering. There is only the sound of natural joy. Therefore, that country is called Ultimate Bliss.”
*   “If one awakens the Bodhi mind, single-mindedly recites Amitabha Buddha, cultivates various merits, and vows to be born in that land, at the end of their life, Amitabha Buddha and His assembly of sages will appear before them. In an instant, they will follow that Buddha and be reborn in his country.”
*   “If there are sentient beings who wish to see Amitabha Buddha in this lifetime, they should awaken the unsurpassed Bodhi mind. They should also single-mindedly recite the name of the land of Ultimate Bliss, accumulate good roots, and dedicate them. By doing so, they will see the Buddha, be born in that land, attain non-retrogression, and achieve unsurpassed enlightenment.”
*   “You must awaken the unsurpassed Bodhi mind and single-mindedly recite Amitabha Buddha.”
*   “You must be filial and sincerely faithful. Believe in the profound words of the Buddha’s sutras and believe that good deeds bring good fortune.”
*   “All who are reborn attain non-retrogression, possess the thirty-two marks of a Buddha, and will all become Buddhas.”
*   “There are limitless, countless, inconceivable, incomparable, and boundless worlds where all the Buddhas and Tathagatas praise the merits of Amitabha Buddha.”
*   “The Buddha's original vows have the power that upon hearing his name, those who wish to be reborn in that land will naturally attain non-retrogression.”
*   “Worldly men and women, if they are in sudden distress, should take refuge in Avalokiteshvara Bodhisattva. There is no one who will not be liberated.”
*   “In the eight directions and above and below, there are countless Buddha lands. Amitabha’s land is vast, long-lasting, bright, and happy. It is supremely unique.”
*   “The secret treasures of all Buddhas are completely understood.”
*   “People in the world contend for unimportant things. In this extremely evil and suffering world, they diligently work to sustain themselves. The high and low, rich and poor, young and old, men and women, are constantly thinking and worrying, driven by their minds. Without fields, they worry about fields. Without homes, they worry about homes. They worry about whether they have or do not have families and possessions.”
*  “People, amidst their cravings and desires, are born alone, die alone, leave alone, and return alone. They experience suffering and joy alone. No one can replace them.”
*  “Why not strive to practice good when you are strong and healthy? What are you waiting for?”
*   “There are many who are deluded, and few who are enlightened. Each harbors malicious intentions. Evil permeates the world. They recklessly act against the way of heaven and earth.”
*   “Love, desire, and glory are not permanent. All will be separated. There is nothing to rejoice in. You should strive diligently to be born in the land of peace and bliss.”
*   “Between heaven and earth, things happen naturally. Even if retribution is not immediate, good and evil will always return in due time.”
*   “Between heaven and earth, the five paths of rebirth are clear. Good and evil deeds have their corresponding retributions. Misfortunes and blessings follow one another. Everyone must bear their own consequences. No one can take their place.”
*   “What is paramount? One should correct one’s own mind. One should correct one’s own body. The ears, eyes, mouth, and nose should all be corrected. Keep one’s mind and body pure, aligning with goodness.”
*   “Know that doubt is a great harm to all Bodhisattvas. It leads to the loss of great benefits. Therefore, one should clearly believe in the unsurpassed wisdom of all Buddhas.”
*   “If you use formless wisdom to cultivate roots of merit, purify the body and mind, separate from discrimination, seek rebirth in the Pure Land, aspire to Buddhahood, you will be born in a Buddha land and attain eternal liberation.”
*  “If good men and women hear the name of Amitabha Buddha and can generate a single thought of joyful fondness, take refuge in Him, respectfully worship Him, and practice as taught, know that these people have gained great benefit and have acquired the merits spoken of above. Their minds will not be inferior or arrogant. All of their good roots will increase. Know that such people are not of the Small Vehicle. In my Dharma, they are called the foremost disciples.”

**The Contemplation Sutra (Buddha Speaks of Contemplation of Immeasurable Life)**

*   "Those who wish to be born in that land should cultivate three types of good conduct."
    *  "First: Filial piety towards parents, serving teachers and elders, compassionately refraining from killing, and cultivating the ten good deeds."
    *  "Second: Accepting and upholding the three refuges, fulfilling all precepts, and not violating decorum."
    *  "Third: Awakening the Bodhi mind, deeply believing in karma, reciting the Mahayana sutras, and encouraging others to practice."

These three actions are called the pure karma. These three types of actions are the pure causes and conditions for the three Buddhas of the past, present, and future.

*   “All Buddhas and Tathagatas are the Dharmakaya body, pervading the minds and thoughts of all sentient beings. Therefore, when you think of the Buddha, that thought is the thirty-two marks and eighty minor marks of a Buddha. That mind becomes the Buddha. That mind is the Buddha.”
*   "The mind of the Buddha is great compassion. With boundless compassion, it embraces all sentient beings.”
*   "Those who see Immeasurable Life Buddha will also see all the immeasurable Buddhas of the ten directions."
*   “Reciting Namo Amitabha Buddha, because you recite the Buddha's name, in each moment, you eliminate the karmic hindrances of eighty billion kalpas of birth and death.”
*   “If a good man or good woman, upon hearing the name of the Buddha and the names of the two Bodhisattvas, eliminate limitless kalpas of sins. How much more so by being mindful of them! If one is mindful of the Buddha, know that this person is like a lotus flower among humans.”
*  “You must remember these words. To remember these words is to remember the name of Immeasurable Life Buddha.”

**The Great Strength Bodhisattva Chapter on Perfect Enlightenment through Mindfulness of the Buddha (From the Shurangama Sutra)**

*   “If sentient beings are mindful of the Buddha, they will surely see the Buddha in the present and in the future. They will not be far from the Buddha. Without relying on expedient methods, their minds will be opened naturally.”
*   “When asked about perfect enlightenment, I have no choice. By gathering the six senses and maintaining pure mindfulness continuously, I have achieved samadhi. This is foremost.”

**The Universal Worthy Bodhisattva’s Conduct and Vows Chapter (From the Flower Garland Sutra)**

*   **The Ten Great Vows of Universal Worthy Bodhisattva**
    1.  Bowing in Reverence to All Buddhas
    2.  Praising the Tathagatas
    3.  Making Extensive Offerings
    4.  Repenting of Karmic Hindrances
    5.  Rejoicing in the Merits of Others
    6.  Requesting the Turning of the Dharma Wheel
    7.  Requesting the Buddhas to Remain in the World
    8.  Constantly Following the Teachings of the Buddha
    9.  Always Accord with Sentient Beings
    10. Dedicating All Merits Universally

*   “When repenting of karmic hindrances, the Bodhisattva reflects: ‘In countless kalpas in the past, due to greed, anger, and ignorance, through body, speech, and mind, I have committed limitless, countless evil deeds. If these evil deeds had a physical form, the entire space of the universe could not contain them. Now, I will, with my pure three karmas, repent sincerely before all Buddhas and Bodhisattvas in the cosmos and all realms of infinite particles, vowing never to repeat them and to always uphold pure precepts.’”
*   "If Bodhisattvas can accord with sentient beings, then they are according with offering to all Buddhas. If one respects and serves sentient beings…"

This translation seeks to be both accurate and easily understood by a contemporary English-speaking audience while preserving the spirit and essence of the original Buddhist text.


In [16]:
translated_texts = [result for result in results]

# join the results together by double whitespace
translated_book = "\n\n".join(translated_texts)

# save the translated book
with open(f"translated_book1.txt", "w") as f:
  f.write(translated_book)

## Next steps

Now you know how to properly prepare data, check token limitations and translate each chunk to obtain your results. Feel free to try this process with a book that interests you!